In [1]:
%load_ext autoreload
%autoreload 2

import logging
from urllib.parse import urljoin
from io import StringIO
from typing import Sequence
from datetime import date, datetime

import requests
import pandas as pd
import numpy as np

from etf_scraper.base import SecurityListing
from etf_scraper.scrapers import ISharesListings, SSGAListings

logger = logging.getLogger(__name__)


ssga_listings = SSGAListings.retrieve_listings()        
ishares_listings = ISharesListings.retrieve_listings()

#TODO: Vanguard, Invesco - get proof of concept for one of ishares, ssga holdings first

/home/nikul/Documents/Programming/ETF_Scraper/src/etf_scraper/scrapers.py:286: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  ssga_web_data_df_.loc[:, "inception_date"] = ssga_web_data_df_[
/home/nikul/Documents/Programming/ETF_Scraper/src/etf_scraper/scrapers.py:289: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  ssga_web_data_df_.loc[:, "net_assets"] = ssga_web_data_df_["net_assets"].apply(
/home/nikul/Documents/Programming/ETF_Scraper/src/etf_scraper/scrapers.py:98: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set th

In [2]:
ssga_listings.tail()

,fund_name,ticker,product_url,inception_date,region,net_assets,exchange,fund_type,asset_class,cusip,isin,benchmark,provider
205,The Industrial Select Sector SPDR® Fund,XLI,https://www.ssga.com/us/en/intermediary/etfs/f...,1998-12-16,US,1.492249e+10,NYSE ARCA EXCHANGE,ETF,Equity,81369Y704,US81369Y7040,Industrial Select Sector Index,SSGA
206,The Materials Select Sector SPDR® Fund,XLB,https://www.ssga.com/us/en/intermediary/etfs/f...,1998-12-16,US,5.739410e+09,NYSE ARCA EXCHANGE,ETF,Equity,81369Y100,US81369Y1001,Materials Select Sector Index,SSGA
207,The Real Estate Select Sector SPDR® Fund,XLRE,https://www.ssga.com/us/en/intermediary/etfs/f...,2015-10-07,US,5.051460e+09,NYSE ARCA EXCHANGE,ETF,Equity,81369Y860,US81369Y8600,Real Estate Select Sector Index,SSGA
208,The Technology Select Sector SPDR® Fund,XLK,https://www.ssga.com/us/en/intermediary/etfs/f...,1998-12-16,US,3.860053e+10,NYSE ARCA EXCHANGE,ETF,Equity,81369Y803,US81369Y8030,Technology Select Sector Index,SSGA
209,The Utilities Select Sector SPDR® Fund,XLU,https://www.ssga.com/us/en/intermediary/etfs/f...,1998-12-16,US,1.668948e+10,NYSE ARCA EXCHANGE,ETF,Equity,81369Y886,US81369Y8865,Utilities Select Sector Index,SSGA


In [72]:
# ticker = 'SPY'.lower()
# url = (
#     "https://www.ssga.com/us/en/intermediary/etfs/library-content/products/fund-data/"
#     f"etfs/us/holdings-daily-us-en-{ticker}.xlsx"
# )
# # ticker = 'SSFOX'.lower()
# # url = (
# #     "https://www.ssga.com/us/en/intermediary/ic/library-content/products/fund-data/"
# #     f"mf/us/holdings-monthly-us-en-{ticker}.xlsx"
# # )
# resp = requests.get(url)
# resp.raise_for_status()

# df = pd.read_excel(ef)

In [32]:
spy = SSGAListings.retrieve_holdings_("SPY")
spy

Ticker Symbol                  SPY
Holdings         As of 10-Jan-2023
nan                            NaN
Name: SPDR® S&P 500® ETF Trust, dtype: object
3


,name,ticker,cusip,sedol,weight,Sector,amount,local_currency,fund_ticker,as_of_date
4,Apple Inc.,AAPL,03783310,2046251,5.957167,Information Technology,166212290.000,USD,SPY,2023-01-10
5,Microsoft Corporation,MSFT,59491810,2588173,5.198587,Information Technology,82857736.000,USD,SPY,2023-01-10
6,Amazon.com Inc.,AMZN,02313510,2000019,2.430652,Consumer Discretionary,98652180.000,USD,SPY,2023-01-10
7,Berkshire Hathaway Inc. Class B,BRK.B,08467070,2073390,1.736894,Financials,20026460.000,USD,SPY,2023-01-10
8,Alphabet Inc. Class A,GOOGL,02079K30,BYVY8G0,1.609392,Communication Services,66391108.000,USD,SPY,2023-01-10
...,...,...,...,...,...,...,...,...,...,...
503,Lincoln National Corp,LNC,53418710,2516378,0.014801,Financials,1716026.000,USD,SPY,2023-01-10
504,DaVita Inc.,DVA,23918K10,2898087,0.013494,Health Care,616935.000,USD,SPY,2023-01-10
505,Fox Corporation Class B,FOX,35137L20,BJJMGY5,0.013357,Communication Services,1625095.000,USD,SPY,2023-01-10
506,DISH Network Corporation Class A,DISH,25470M10,2303581,0.011122,Communication Services,2735517.000,USD,SPY,2023-01-10


In [ ]:
'Weight (%)'

In [33]:
spy['amount']

4      166212290.000
5       82857736.000
6       98652180.000
7       20026460.000
8       66391108.000
           ...      
503      1716026.000
504       616935.000
505      1625095.000
506      2735517.000
507      1354944.000
Name: amount, Length: 504, dtype: object

In [29]:
with open("test_SSGA_SPY_resp.xlsx", 'wb') as f:
    f.write(spy.content)

In [20]:
fhun = ishares_listings[ishares_listings['fund_name'].str.contains('500')].iloc[0]
fhun['ticker']

'IVV'

In [34]:
fhun_now = ISharesListings.retrieve_holdings_(fhun['ticker'], fhun['product_url'], None)
fhun_now

Index(['Ticker', 'Name', 'Sector', 'Asset Class', 'Market Value', 'Weight (%)',
       'Notional Value', 'Shares', 'Price', 'Location', 'Exchange', 'Currency',
       'FX Rate', 'Market Currency', 'Accrual Date'],
      dtype='object')


,ticker,name,sector,asset_class,market_value,weight,notional_value,amount,price,location,exchange,currency,fx_rate,market_currency,Accrual Date,fund_ticker,as_of_date
0,AAPL,APPLE INC,Information Technology,Equity,1.789358e+10,6.00,1.789358e+10,134044353.0,133.49,United States,NASDAQ,USD,1.0,USD,NaN,IVV,None
1,MSFT,MICROSOFT CORP,Information Technology,Equity,1.575460e+10,5.28,1.575460e+10,66821890.0,235.77,United States,NASDAQ,USD,1.0,USD,NaN,IVV,None
2,AMZN,AMAZON COM INC,Consumer Discretionary,Equity,7.565306e+09,2.53,7.565306e+09,79559430.0,95.09,United States,NASDAQ,USD,1.0,USD,NaN,IVV,None
3,BRKB,BERKSHIRE HATHAWAY INC CLASS B,Financials,Equity,5.174177e+09,1.73,5.174177e+09,16150628.0,320.37,United States,New York Stock Exchange Inc.,USD,1.0,USD,NaN,IVV,None
4,GOOGL,ALPHABET INC CLASS A,Communication,Equity,4.900160e+09,1.64,4.900160e+09,53541963.0,91.52,United States,NASDAQ,USD,1.0,USD,NaN,IVV,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,NWS,NEWS CORP CLASS B,Communication,Equity,2.093654e+07,0.01,2.093654e+07,1067102.0,19.62,United States,NASDAQ,USD,1.0,USD,NaN,IVV,None
505,ETD_USD,ETD USD BALANCE WITH R93531,Cash and/or Derivatives,Cash,7.500000e+01,0.00,7.500000e+01,75.0,100.00,United States,NaN,USD,1.0,USD,"Jan 01, 2000",IVV,None
506,MARGIN_USD,FUTURES USD MARGIN BALANCE,Cash and/or Derivatives,Cash Collateral and Margins,-7.500000e+01,0.00,-7.500000e+01,-75.0,100.00,United States,NaN,USD,1.0,USD,"Jan 01, 2000",IVV,None
507,ESH3,S&P500 EMINI MAR 23,Cash and/or Derivatives,Futures,0.000000e+00,0.00,6.629385e+08,3323.0,3990.00,NaN,Index And Options Market,USD,1.0,USD,NaN,IVV,None


In [36]:
fhun_now['amount'].sum()

3511335561.0

In [30]:
fhun_ = ISharesListings.retrieve_holdings_(fhun['ticker'], fhun['product_url'], None) # date(2022, 12, 30)
fhun_

<Response [200]>

In [31]:
with open("./test_IShares_IVV_holdings_current_resp.csv", 'wb') as f:
    f.write(fhun_.content)

In [9]:
# # query_dates = pd.bdate_range('2022-01-01', '2023-01-01')

# dfs = {}

# for query_date in query_dates[:10]:
#     r = get_holdings(query_date)
#     dfs[query_date] = pd.read_csv(StringIO(r.content.decode()), skiprows=9, skipfooter=2)

In [113]:
from datetime import datetime

from etf_scraper.utils import check_missing_cols
from etf_scraper.base import ProviderListings

class VanguardListings(ProviderListings):
    provider = "Vanguard"
    req_user_header ={
        "user-agent": 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    } # need to pass otherwise requests will be denied
    listing_endpoint = 'https://investor.vanguard.com/investment-products/list/funddetail'
    listing_resp_mapping = {
        "ticker": "ticker",
        "cusip": "cusip",
        "longName": "fund_name",
        "inceptionDate": "inception_date",
        "fundId": "product_id",
        "style": "asset_class",
        "type": "subasset_class",
    }    
    fund_url = "https://advisors.vanguard.com/investments/products/{}"


    @classmethod
    def retrieve_listings(cls):
        """
        #TODO: missing important info: eg AUM, country etf - not possible to get without visiting individual
        fund pages
        #FIXME: duplicates via multiple share classes, eg VFIAX, VFFSX, VOO
        """
        resp = requests.get(cls.listing_endpoint, headers=cls.req_user_header)
        resp.raise_for_status()
        fund_data = resp.json()
        fund_list = [x['profile'] for x in fund_data['fund']['entity']]
        fund_df = pd.DataFrame(fund_list)
        
        check_missing_cols(['ticker'], fund_df.columns)
        
        fund_df_ = fund_df.dropna(subset='ticker').rename(columns=cls.listing_resp_mapping)
        
        def map_fund_type(fund_row : pd.Series):
            if fund_row['isETF']:
                return "ETF"
            elif fund_row['isMutualFund']:
                return "MF"
            return np.nan
        
        fund_types = fund_df_.apply(map_fund_type, axis=1)
        
        fund_df_ = fund_df_.reindex(columns=list(cls.listing_resp_mapping.values()))
        
        fund_df_.loc[:, "fund_type"] = fund_types
        
        map_to_url = lambda x: cls.fund_url.format(x.upper())
        fund_df_.loc[:, "product_url"] = fund_df_['ticker'].apply(map_to_url)
        
        def parse_inception_date(x):
            try:
                return datetime.fromisoformat(x).date()
            except:
                return pd.NaT

        fund_df_.loc[:, "inception_date"] = fund_df_['inception_date'].apply(parse_inception_date)
        fund_df_.loc[:, "provider"] = cls.provider
        
        return fund_df_
    
    # @classmethod
    # def _parse_holdings_resp()
    
    @classmethod
    def retrieve_holdings_(cls, ticker : str, )
        

In [117]:
u = "https://eds.ecs.gisp.c1.vanguard.com/eds-eip-distributions-service/holdings/holding-details-history/3145.json?as-of-date=2022-10-31"
r = requests.get(u, headers=VanguardListings.req_user_header)
r.raise_for_status()
r_ = r.json()

In [118]:
r_

[{'portId': '3145',
  'accountingTypeCode': 23,
  'holdingTypeCode': 'ORIG',
  'holdingDetailItem': [{'effectiveDate': '2022-10-31',
    'name': 'US 5YR NOTE (CBT) Dec22',
    'quantity': 3226.0,
    'mktValPercent': 0.7590829479,
    'salesPosition': 'LONG',
    'bticker': 'FVZ2',
    'issueTypeCode': '50201',
    'parentIssueTypeCode': '5',
    'issueTypeName': 'Physical Commodity Future',
    'parentIssueTypeName': 'Futures',
    'market': '',
    'marketVal': 343871437.5,
    'reportingName': 'US 5YR NOTE (CBT) Dec22'},
   {'effectiveDate': '2022-10-31',
    'name': 'MKTLIQ',
    'quantity': 1887634.24,
    'mktValPercent': 0.4166046466,
    'salesPosition': 'LONG',
    'issueTypeCode': '102',
    'parentIssueTypeCode': '1',
    'issueTypeName': 'Mutual Fund',
    'parentIssueTypeName': 'Equity',
    'market': 'US',
    'marketVal': 188725671.02,
    'CUSIP': 'CMT001142',
    'reportingName': 'Vanguard Market Liquidity Fund'},
   {'effectiveDate': '2022-10-31',
    'name': 'US 2YR 

In [114]:
van = VanguardListings.retrieve_listings()

In [115]:
van

,ticker,cusip,fund_name,inception_date,product_id,asset_class,subasset_class,fund_type,product_url,provider
0,BSV,921937827,Vanguard Short-Term Bond ETF,2007-04-03,0924,Bond Funds,Short-Term Bond,ETF,https://advisors.vanguard.com/investments/prod...,Vanguard
1,VCSH,92206C409,Vanguard Short-Term Corporate Bond ETF,2009-11-19,3145,Bond Funds,Short-Term Bond,ETF,https://advisors.vanguard.com/investments/prod...,Vanguard
2,VGSH,92206C102,Vanguard Short-Term Treasury ETF,2009-11-19,3142,Bond Funds,Short-Term Bond,ETF,https://advisors.vanguard.com/investments/prod...,Vanguard
3,VUSB,92203C303,Vanguard Ultra-Short Bond ETF,2021-04-05,0931,Bond Funds,Short-Term Bond,ETF,https://advisors.vanguard.com/investments/prod...,Vanguard
4,VCEB,921910691,Vanguard ESG U.S. Corporate Bond ETF,2020-09-22,4158,Bond Funds,Intermediate-Term Bond,ETF,https://advisors.vanguard.com/investments/prod...,Vanguard
...,...,...,...,...,...,...,...,...,...,...
342,VTPSX,921909776,Vanguard Total International Stock Index Fund ...,2010-11-30,1870,Stock Funds,International/Global Stock,MF,https://advisors.vanguard.com/investments/prod...,Vanguard
343,VTSNX,921909784,Vanguard Total International Stock Index Fund ...,2010-11-29,1869,Stock Funds,International/Global Stock,MF,https://advisors.vanguard.com/investments/prod...,Vanguard
344,VTWAX,922042619,Vanguard Total World Stock Index Fund Admiral ...,2019-02-07,5028,Stock Funds,International/Global Stock,MF,https://advisors.vanguard.com/investments/prod...,Vanguard
345,VTWIX,922042759,Vanguard Total World Stock Index Fund Institut...,2008-10-09,0826,Stock Funds,International/Global Stock,MF,https://advisors.vanguard.com/investments/prod...,Vanguard


In [63]:
fund_list = fund_data['fund']['entity']
fund_list_ = [x['profile'] for x in fund_list]

In [72]:
# pd.DataFrame(fund_list_).columns
fund_list_[0]

{'fundId': '0924',
 'ticker': 'BSV',
 'instrumentId': 2845959,
 'shortName': 'Short-Term Bond ETF',
 'longName': 'Vanguard Short-Term Bond ETF',
 'cusip': '921937827',
 'IOVTicker': 'BSV.IV',
 'inceptionDate': '2007-04-03T00:00:00-04:00',
 'newspaperAbbreviation': 'Short-Term Bond     ',
 'style': 'Bond Funds',
 'type': 'Short-Term Bond',
 'category': 'Short-Term Bond',
 'customizedStyle': 'Bond - Short-term Investment',
 'fixedIncomeInvestmentStyleId': '1',
 'fixedIncomeInvestmentStyleName': 'Short-term Treasury',
 'secDesignation': '',
 'maximumYearlyInvestment': '',
 'expenseRatio': '0.0400',
 'expenseRatioAsOfDate': '2022-04-29T00:00:00-04:00',
 'isInternalFund': True,
 'isExternalFund': False,
 'isMutualFund': False,
 'isETF': True,
 'isVLIP': False,
 'isVVAP': False,
 'is529': False,
 'hasAssociatedInvestorFund': True,
 'hasMoreThan1ShareClass': True,
 'isPESite': False,
 'fundFact': {'isActiveFund': True,
  'isClosed': False,
  'isClosedToNewInvestors': False,
  'isFundOfFunds':

In [57]:
fund_data['fund']['entity'][0]['profile']

{'fundId': '0924',
 'ticker': 'BSV',
 'instrumentId': 2845959,
 'shortName': 'Short-Term Bond ETF',
 'longName': 'Vanguard Short-Term Bond ETF',
 'cusip': '921937827',
 'IOVTicker': 'BSV.IV',
 'inceptionDate': '2007-04-03T00:00:00-04:00',
 'newspaperAbbreviation': 'Short-Term Bond     ',
 'style': 'Bond Funds',
 'type': 'Short-Term Bond',
 'category': 'Short-Term Bond',
 'customizedStyle': 'Bond - Short-term Investment',
 'fixedIncomeInvestmentStyleId': '1',
 'fixedIncomeInvestmentStyleName': 'Short-term Treasury',
 'secDesignation': '',
 'maximumYearlyInvestment': '',
 'expenseRatio': '0.0400',
 'expenseRatioAsOfDate': '2022-04-29T00:00:00-04:00',
 'isInternalFund': True,
 'isExternalFund': False,
 'isMutualFund': False,
 'isETF': True,
 'isVLIP': False,
 'isVVAP': False,
 'is529': False,
 'hasAssociatedInvestorFund': True,
 'hasMoreThan1ShareClass': True,
 'isPESite': False,
 'fundFact': {'isActiveFund': True,
  'isClosed': False,
  'isClosedToNewInvestors': False,
  'isFundOfFunds':

In [56]:
"https://investor.vanguard.com/investment-products/mutual-funds/profile/" + fund_data['fund']['entity'][0]['profile']['ticker'].lower()

'https://investor.vanguard.com/investment-products/mutual-funds/profile/bsv'